In [6]:
import os  
import requests
import json
import base64
from openai import AzureOpenAI  
from dotenv import load_dotenv

load_dotenv() # Load environment variables from .env file
def request_gpt(user_text):
    endpoint = os.getenv("ENDPOINT_URL")
    deployment = os.getenv("DEPLOYMENT_NAME")
    search_endpoint = os.getenv("SEARCH_ENDPOINT")
    search_key = os.getenv("SEARCH_KEY")
    search_index = os.getenv("SEARCH_INDEX_NAME")
    api_key = os.getenv("AZURE_OPENAI_KEY")

    api_version = "2024-05-01-preview"
    url = f"{endpoint}openai/deployments/{deployment}/chat/completions?api-version={api_version}"
    
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key,
    }

    body = {
        "messages": [
            {"role": "system", "content": "너는 부동산 및 청년 주택 매매·임대 관련 공문을 기반으로 정보를 제공하는 AI야. 가능한 한 한 개의 문서만 참조해서 실용적이고 날짜 중심의 정보를 표로 정리해줘."},
            {"role": "user", "content": user_text}
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "max_tokens": 1000,
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": search_endpoint,
                    "index_name": search_index,
                    "semantic_configuration": "default",
                    "query_type": "vector_semantic_hybrid",
                    "in_scope": True,
                    "top_n_documents": 1,
                    "strictness": 3,
                    "authentication": {
                        "type": "api_key",
                        "key": search_key
                    },
                    "embedding_dependency": {
                        "type": "integrated"
                    }
                }
            }
        ]
    }

    response = requests.post(url, headers=headers, data=json.dumps(body))
    
    if response.status_code == 200:
        result = response.json()
        return result["choices"][0]["message"]["content"]
    else:
        return f"요청 실패: {response.status_code} - {response.text}"

def main():
    print("Azure OpenAI 부동산 상담 AI (종료하려면 'exit' 또는 '나가기' 입력)")
    print("-----------------------------------------------------")
    
    while True:
        user_input = input("질문: ")
        if user_input.lower() in ['exit', '나가기']:
            print("프로그램을 종료합니다.")
            break
            
        print("\n처리 중...\n")
        response = request_gpt(user_input)
        print(f'질문: {user_input}')
        print('-' * 50)
        print(f'답변: {response}')
        print('*' * 50)
        print()

if __name__ == "__main__":
    main()

Azure OpenAI 부동산 상담 AI (종료하려면 'exit' 또는 '나가기' 입력)
-----------------------------------------------------

처리 중...

질문: 안녕
--------------------------------------------------
답변: 안녕하세요! 무엇을 도와드릴까요? 😊
**************************************************


처리 중...

질문: 모집 안내에 대해 알려줘
--------------------------------------------------
답변: 다음은 제공된 문서를 기반으로 한 2025년도 1차 청년 매입 임대 입주자 모집 안내의 주요 정보를 정리한 표입니다:

| 항목                 | 세부 내용                                                                                       |
|---------------------|---------------------------------------------------------------------------------------------|
| **모집 대상**        | 청년(19~39세), 대학생 및 취업준비생                                                              |
| **입주 기준**        | 시중시세 40~50% 수준으로 임대                                                                  |
| **모집 방법**        | - LH 마이홈센터(1600-1004)를 통해 세부 내용 안내 및 상담 가능<br>- 1인 1주택 신청 원칙, 중복 신청 불가              |
| **모집 공고일**      | 2025년 3월 27일 (목요일)     